## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

## Tutorial: 

#### Configure the Environment

In [ ]:
import json
import os
import chi

#Config with your project and site
chi.set('project_name', 'chameleon')   # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'
chi.use_site('CHI@UC')               # Authenticate to CHI@UC site

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_VLANTutorial_"
server_name = prefix+'Server'
network_name = prefix+'Net'
subnet_name = prefix+'Subnet'
router_name = prefix+'Router'
lease_name = prefix+'Lease'

#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_skylake"
server_count=1
physical_network="physnet1"

#### Create a Lease

In [ ]:
import chi.lease
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
chi.lease.add_fip_reservation(reservation_list, count=1)

# Create the lease
lease = chi.lease.create_lease(lease_name,
                       reservations=reservation_list,
                       start_date=start_date,
                       end_date=end_date)
    
#Print the lease info
print(json.dumps(lease, indent=2))

#### Get the Reservations

In [ ]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
network_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Configute the Network

In [ ]:
import chi.network

network = chi.network.get_network(network_name)
network_id = network['id']

print(f'Network ID: {network_id}')

In [ ]:
subnet = chi.network.create_subnet(subnet_name, network_id)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router_by_name(router_name, subnet_name)

#### Start the Server

In [ ]:
import chi.server

# Create the server
server = chi.server.create_server(server_name, 
                                  reservation_id=compute_reservation_id, 
                                  network_name=network_name, 
                                  image_name=image_name)
# Wait until the server is active
chi.server.wait_for_active(server.id)

#### Associate the Floating IP

In [ ]:
floating_ip = chi.server.associate_floating_ip(server.id)

print(f'Floating IP: {floating_ip}')

#### Delete Server

In [ ]:
chi.server.delete_server(server.id)

#### De-configure Network

In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

#### Release Lease

In [ ]:
chi.lease.delete_lease(lease['id'])